In [1]:
import pulp
from haversine import haversine


full_tsp=[[0,(90,0),0],
[48639,(69.883082808,-104.811538989),24.3631662716],
[8294,(67.9234855123,-104.847795642),6.07971814328],
[78916,(63.9484583342,-104.918358181),50],
[21860,(63.9335695509,-104.991654862),9.59724841331],
[28053,(63.4227158928,-104.863163805),1],
[35053,(62.9888439249,-104.929555462),31.1761722349],
[28125,(62.5641940328,-104.925905611),9.36404536361],
[77500,(60.7912673657,-104.830030381),10.9372824279],
[66257,(57.892420319,-104.927282344),4.43021187589],
[77562,(57.8839108063,-104.831342915),13.932694415],
[32058,(57.788360065,-104.895752253),16.7583199142],
[10489,(57.6983991796,-104.818866286),1],
[32506,(56.4996044858,-104.960679825),1],
[2779,(56.3950925722,-104.914964473),9.280046014],
[78496,(54.0234609634,-104.937615867),25.7735858718],
[90135,(53.747731232,-104.822994369),1],
[88470,(53.497658807,-104.947592945),1],
[68638,(53.488777969,-104.823830626),26.4501222546],
[12789,(52.7998332273,-104.933660193),9.84046118901],
[34909,(52.0198128889,-104.927654096),1],
[33140,(51.6307109197,-104.917628688),1],
[95155,(50.8224495388,-104.939181748),10.1027925324],
[44010,(50.21002478,-104.901468326),35.588750056],
[76584,(50.0717087129,-104.949111062),26.7539242891],
[88376,(49.2545614241,-104.913670707),9.62041879373],
[53070,(49.1221499413,-104.851753298),4.51808035851],
[37429,(48.2689986402,-104.905888294),1],
[86997,(44.3768867381,-104.868489288),1],
[14492,(43.0634537517,-104.918113873),3.78935363199],
[89764,(42.8845537903,-104.933694487),1.74329021981],
[82959,(41.613120493,-104.942576755),11.553456657],
[58287,(41.0383813466,-104.935915753),15.4080036368],
[81913,(40.8491273658,-104.915963856),1],
[40842,(40.6377907524,-104.9035771),1],
[57677,(39.4926050894,-104.888962183),24.3532949554],
[52514,(38.9148253286,-104.889588033),26.6939321834],
[98122,(37.8722791176,-104.922882693),25.1925367773],
[59301,(37.1627002304,-104.869496198),35.2131348147],
[53491,(36.6293305743,-104.970495506),27.4724261824],
[64377,(35.7680503514,-104.946686574),11.9341865177],
[77778,(35.3790319837,-104.962116072),50],
[17630,(35.181337763,-104.88074419),1],
[6141,(35.0624354701,-104.91266238),17.8554559607],
[26103,(34.9443275631,-104.810615261),1.01931512925],
[84541,(34.9313579702,-104.894748311),5.30845904622],
[35415,(33.2863171191,-104.962355603),26.6760903357],
[68189,(32.8177960753,-104.816220701),42.8712978126],
[58303,(31.5373914723,-104.883909255),27.5195395206],
[27314,(31.2804084063,-104.902449714),28.3949391754],
[62314,(29.7266081556,-104.92712045),1],
[47876,(29.4620582563,-104.886835764),21.1919158115],
[27894,(29.0364758783,-104.86314241),31.6423167702],
[16268,(28.9277934237,-104.855463674),1],
[65166,(28.835340498,-104.874709328),1],
[48781,(28.5765558307,-104.970341498),7.02087894132],
[2334,(27.3452433851,-104.831470351),1],
[97358,(26.7559366542,-104.942354039),29.9243802919],
[31570,(25.9626529866,-104.961333137),4.47309790099],
[9135,(25.4497888069,-104.825553875),2.31168600177],
[13864,(24.3654728834,-104.9627218),1],
[81837,(23.1811410697,-104.939536933),21.940126991],
[75883,(23.1622088963,-104.915464734),11.9951851864],
[66090,(22.692575137,-104.881243799),1],
[35487,(21.9184104502,-104.865175522),28.2981320476],
[31962,(21.8718327884,-104.839017652),1],
[34236,(21.6300946599,-104.81218227),10.4776268661],
[21938,(20.4189116219,-104.820759453),24.7082431869]]


#tsp=full_tsp[11:20]

tsp=full_tsp

ncities=len(tsp)


C=[haversine(tsp[i][1],tsp[j][1]) for i in range(ncities) for j in range(ncities)]

wgt=[tsp[i][2] for i in range(ncities)]

W=sum(wgt)
wf=10

north_pole=(90,0)

def path_opt_test(llo):
    f_ = 0.0
    d_ = 0.0
    l_ = north_pole
    for i in range(len(llo)):
        d_ += haversine(l_, llo[i][1])
        f_ += d_ * llo[i][2]
        l_ = llo[i][1]
    d_ += haversine(l_, north_pole)
    f_ += d_ * 10 #sleigh weight for whole trip
    return f_

print path_opt_test(tsp)


5184399.57089


In [2]:
Large=10**9 #Large Number

cities=range(ncities*ncities)

Srange=range(ncities*ncities*ncities)

xx=[0 for i in range(ncities) for j in range(ncities)]
for i in range(ncities-1):
    xx[ncities*i+i+1]=1

xx[ncities*(ncities-1)]=1


##ith city at kth position in tour
R = pulp.LpVariable.dicts('trips', cities,lowBound=0, upBound=1, cat='Integer')

##Edge Weight for tour i to j at kth position (sequence dependent due to weight drop offs)
D = pulp.LpVariable.dicts('arcs', Srange,lowBound=0.0, upBound=Large, cat='Continuous')

tsp_model = pulp.LpProblem("TSP Model", pulp.LpMinimize)

##Quadratic Objective
tsp_model += sum([D[ncities*ncities*i+ncities*j+k] for i in range(ncities) for j in range(ncities) for k in range(ncities)]) 

#tsp_model += sum([C[ncities*i+j]*S[ncities*ncities*i+ncities*j+k] * (W + wf - sum(wgt[l]*R[ncities*l+m] for l in range(ncities) for m in range(min(k+1,ncities-1)))) for i in range(ncities) for j in range(ncities) for k in range(ncities-1)]) 

##Relationship of D with Distance and Weights
for i in range(ncities):
    for j in range(ncities):
        for k in range(ncities):
            if k<(ncities-1):
                tsp_model += D[ncities*ncities*i+ncities*j+k]>=C[ncities*i+j] * (W + wf - sum([wgt[l]*R[ncities*l+m] for l in range(ncities) for m in range(k)])) - (1-R[ncities*i+k])*Large - (1-R[ncities*j+k+1]) *Large
            else:
                tsp_model += D[ncities*ncities*i+ncities*j+k]>=C[ncities*i+j] * (W + wf - sum([wgt[l]*R[ncities*l+m] for l in range(ncities) for m in range(k)])) - (1-R[ncities*i+k])*Large - (1-R[ncities*j]) * Large
#for i in range(ncities):
#    for j in range(ncities):
#        tsp_model += D[ncities*ncities*i+ncities*j]==C[ncities*i+j] * (W + wf)


##Constraints on R
for i in range(ncities):
    tsp_model += sum([R[ncities*i+j] for j in range(ncities)])==1

for j in range(ncities):
    tsp_model += sum([R[ncities*i+j] for i in range(ncities)])==1

##Start at North pole
for i in range(ncities):
    tsp_model += R[0]==1

##Relationships between D and R

##Includes final route as well
for i in range(ncities):
    for j in range(ncities):
        for k in range(ncities):
            tsp_model += D[ncities*ncities*i+ncities*j+k] <= R[ncities*i+k] *Large
            

for i in range(ncities):
    for j in range(ncities):
        for k in range(ncities-1):
            tsp_model += D[ncities*ncities*i+ncities*j+k] <= R[ncities*j+k+1] *Large


for i in range(ncities):
    for j in range(ncities):
        tsp_model += D[ncities*ncities*i+ncities*j+ncities-1] <= R[ncities*j] *Large

        
for i in range(ncities):
    for j in range(ncities):
        for k in range(ncities):
            tsp_model += D[ncities*ncities*i+ncities*j+k] >=0.0

#Max 5 changes
tsp_model += sum(R[ncities*i+j]-xx[ncities*i+j] for i in range(ncities) for j in range(ncities))>=-5
tsp_model += sum(R[ncities*i+j]-xx[ncities*i+j] for i in range(ncities) for j in range(ncities))<= 5
                        
#tsp_model.writeLP("tsp_model.lp")

tsp_model.writeLP("tsp_model.lp")

tsp_model.solve(pulp.GLPK(options=['--dual','--bfs','--cuts','--tmlim', '180']))
    
#tsp_model.solve();

KeyboardInterrupt: 

In [58]:
print pulp.LpStatus[tsp_model.status]

# print 10**9
# print W, wf
# print wgt
# print tsp
# print C


for j in range(ncities):
    for i in range(ncities):
        if R[ncities*i+j].value()>0:
            print 'Position '+str(j)+': '+str(i)

print tsp_model.objective.value()

Optimal
Position 0: 0
Position 1: 1
Position 2: 3
Position 3: 2
Position 4: 4
Position 5: 7
Position 6: 8
Position 7: 6
Position 8: 5
40119.73


In [ ]:
import pulp
##Traveling Salesman Problem (Subtour Eliminations)
def tsp_solve_mtz(ll):
    
    locations=[north_pole]+[ll[i][1] for i in range(len(ll))]
    weights=[0]+[ll[i][2] for i in range(len(ll))]
    tot_weight=sum(weights)
    
    ncities=len(locations)
    cities = range(ncities*ncities)
    C=[haversine(locations[i],locations[j])*weights[j] for i in range(ncities) for j in range(ncities)]

    X = pulp.LpVariable.dicts('trips', cities,lowBound=0, upBound=1, cat='Integer')
    
    U = pulp.LpVariable.dicts('seq', sequence,lowBound=0,upBound=ncities-1,cat='Integer')

    tsp_model = pulp.LpProblem("TSP Model", pulp.LpMinimize)

    #Without weights
    #tsp_model += sum([C[i]*X[i] for i in range(len(C))])
    
    #Wrong formulation 1
    #tsp_model += tot_weight*sum([C[i]*X[i] for i in range(len(C))]) - sum(weights[k]*C[ncities*k+j]*X[ncities*k+j] for k in range(1,ncities) for j in range(ncities))

    ##Approximate formulation
    tsp_model += sum([C[ncities*i+j]*(10+tot_weight-weights[i])*X[ncities*i+j] for i in range(ncities) for j in range(ncities)])
    
    for i in range(ncities):
        tsp_model += sum([X[ncities*i+j] for j in range(ncities)])==1

    for j in range(ncities):
        tsp_model += sum([X[ncities*i+j] for i in range(ncities)])==1

    for i in range(ncities):
        tsp_model += X[ncities*i+i]==0
        
    #Subtour elimination and Sequence
    tsp_model += U[0] == 0

    for i in range(1,ncities):
        tsp_model += U[i]>=1

    for i in range(1,ncities):
        for j in range(1,ncities):
            if i<>j:
                tsp_model += U[i]-U[j]+(ncities-1)*X[ncities*i+j] <= (ncities-2)


    tsp_model.writeLP("tsp_model.lp")

    tsp_model.solve(pulp.GLPK(options=['--dual','--bfs','--cuts','--tmlim', '180']))

    print pulp.LpStatus[tsp_model.status]

    ##Re-arrange ll
    lopt=list()
    snode=0
    while True:
        for j in range(ncities):
            if (X[ncities*snode+j].value()>0):
                #print 'Next Node',j
                if j>0:
                    lopt.append(ll[j-1])
                break
        
        snode=j
        if snode==0:
            break
        
    #print lopt
    return lopt